In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score, classification_report
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tabulate import tabulate

In [3]:
# Generate dummy data
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Define MLP model class
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_layers, activation_fn):
        super(MLPModel, self).__init__()
        layers = []
        for neurons in hidden_layers:
            layers.append(nn.Linear(input_size, neurons))
            layers.append(activation_fn())
            input_size = neurons
        layers.append(nn.Linear(input_size, 1))  # Output layer
        layers.append(nn.Sigmoid())  # Sigmoid for binary classification
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Define hyperparameters
hidden_layer_options = [[4], [8], [16], [32], [64], [4, 16], [16, 32], [32, 64], [4, 16, 32], [16, 32, 64]]
activation_functions = [nn.Identity, nn.Sigmoid, nn.ReLU, nn.Tanh]
epoch_options = [1, 10, 25, 50, 100, 250]
learning_rate_options = [10, 1, 0.1, 0.01, 0.001, 0.0001]
batch_size_options = [16, 32, 64, 128, 256, 512]

# Train and evaluate model for each combination
results = []

for hidden_layers in hidden_layer_options:
    for activation_fn in activation_functions:
        for epochs in epoch_options:
            for lr in learning_rate_options:
                for batch_size in batch_size_options:
                    # Initialize model, loss function, and optimizer
                    model = MLPModel(X_train.shape[1], hidden_layers, activation_fn)
                    criterion = nn.BCELoss()
                    optimizer = optim.Adam(model.parameters(), lr=lr)

                    # Training loop
                    model.train()
                    for epoch in range(epochs):
                        for i in range(0, len(X_train_tensor), batch_size):
                            X_batch = X_train_tensor[i:i+batch_size]
                            y_batch = y_train_tensor[i:i+batch_size]

                            optimizer.zero_grad()
                            outputs = model(X_batch)
                            loss = criterion(outputs, y_batch)
                            loss.backward()
                            optimizer.step()

                    # Evaluate the model
                    model.eval()
                    with torch.no_grad():
                        predictions = model(X_test_tensor).round()
                        accuracy = accuracy_score(y_test, predictions.numpy())
                        mae = mean_absolute_error(y_test, predictions.numpy())
                        mse = mean_squared_error(y_test, predictions.numpy())
                        r2 = r2_score(y_test, predictions.numpy())

                    # Store the results
                    results.append({
                        'Hidden_Layers': hidden_layers,
                        'Activation': activation_fn.__name__,
                        'Epochs': epochs,
                        'Learning_Rate': lr,
                        'Batch_Size': batch_size,
                        'Loss': loss.item(),
                        'Accuracy': accuracy,
                        'MAE': mae,
                        'MSE': mse,
                        'R2': r2
                    })

# Convert results to DataFrame and save
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='Accuracy', ascending=False)
results_df.to_csv('MLP_Dummy_Classification_Results.csv', index=False)

# Print top 20 results in tabulated format
print(tabulate(results_df.head(20), headers='keys', tablefmt='grid'))


+------+-----------------+--------------+----------+-----------------+--------------+-------------+------------+-------+-------+----------+
|      | Hidden_Layers   | Activation   |   Epochs |   Learning_Rate |   Batch_Size |        Loss |   Accuracy |   MAE |   MSE |       R2 |
+======+=================+==============+==========+=================+==============+=============+============+=======+=======+==========+
| 4917 | [4, 16]         | ReLU         |      100 |           0.01  |          128 | 0.0425412   |      0.9   | 0.1   | 0.1   | 0.59803  |
+------+-----------------+--------------+----------+-----------------+--------------+-------------+------------+-------+-------+----------+
| 8126 | [16, 32, 64]    | Sigmoid      |       50 |           0.001 |           64 | 0.174808    |      0.895 | 0.105 | 0.105 | 0.577932 |
+------+-----------------+--------------+----------+-----------------+--------------+-------------+------------+-------+-------+----------+
| 5102 | [4, 16]    

In [1]:
model.eval()

# Generate predictions on the test set
with torch.no_grad():
    y_pred = model(X_test_tensor).round().numpy()
    y_true = y_test.values

# Create the classification report
report = classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1'])

# Print the classification report
print("Classification Report:\n")
print(report)


NameError: name 'model' is not defined